In [1]:
import json
from pprint import pprint 
data = []
with open('reviews_Patio_Lawn_and_Garden.json') as f:
    for line in f:
        data.append(json.loads(line))

In [2]:
import numpy as np
yu = data[0].keys()

dt = list()
for l in data:
	lst = []
	for k in [yu[0], yu[1], yu[3], yu[5]]:
		lst.append(str(l[k]))
	dt.append(lst)

dt = np.array(dt)

In [3]:
import ast
helpful_likes = []
for i in dt:
    helpful_likes.append(ast.literal_eval(i[2]))
print helpful_likes[:10]

weighted_scores = []
weights = []
for i in range(len(helpful_likes)):
    rating = float(dt[i][3])
    upvotes = helpful_likes[i][0]
    total_votes = helpful_likes[i][1]
    downvotes = total_votes - upvotes
    weighted_rating = ((1+upvotes)*rating + downvotes*(6-rating))/(1+total_votes)
    weighted_scores.append(weighted_rating)
    weights.append(abs(upvotes+1 - downvotes))

[[6, 7], [0, 0], [9, 11], [1, 1], [2, 2], [6, 8], [0, 0], [1, 6], [1, 1], [15, 16]]


In [5]:
import numpy as np
ratings = dt[:,3]
ratings = np.array(ratings, dtype = np.float32)
print(np.unique(dt[:,1]).shape)			# no of products

weights = np.array(weights, dtype = np.float32)
a = np.asarray([1,2,3,4])
w = np.asarray([1,2,3,4])
print np.average(a,weights=w)

out = open('weighted_mean_PLG.csv', 'w')				# Statistical model 1 : Avg rating
for each_product in np.unique(dt[:,1]):
    product_set = ratings[dt[:,1] == each_product]
    w = weights[dt[:,1] == each_product]
    if sum(w)>0:
        out.write(each_product)
        out.write(';')
        out.write(str(np.average(product_set, weights = w)))
        out.write('\n')
out.close()


(105984,)
3.0


In [14]:
import csv
import numpy as np
import math
import collections


from random import shuffle
from itertools import combinations
import random

n = 100
k = 10
uni , count = np.unique(dt[:,0],return_counts=True)
yu = uni[count>=n]
test_data = []
test_set = []
test_labels = []
training_data = dt
print len(training_data)
print len(yu)
j = 0 
for i in yu:
    indices = np.array([i1 for i1, x in enumerate(training_data[:,0]) if x == i])
    shuffle(indices)
    k_indices = combinations(indices[0:k],2)
    kl = []
    for n in k_indices:
    	kl.append(n)
    shuffle(kl)
    kl = kl[0:(k*k-3)]
    tl = []
    ts = []
    for n in kl:
    	if float(training_data[n[0]][3]) > float(training_data[n[1]][3]):
        	tl.append(training_data[n[0]][1])
        else:
        	tl.append(training_data[n[1]][1])
        ts.append([training_data[n[0]][1],training_data[n[1]][1]])
    test_labels.append(tl)
    test_set.append(ts)
    training_data = np.delete(training_data,indices[0:k],0)
print len(training_data) 
print len(test_set)

def loadClass2(filename):							#to load data from predicted ratings file
	with open(filename, 'r') as f:
		prod = []
		prod_mean = []
		for x in f.readlines():
			tmp = x.split('\n')
			data = tmp[0].split(';')
			prod.append(data[0])
			prod_mean.append(data[1])
		return prod,prod_mean

products, prod_means = loadClass2('weighted_mean_PLG.csv')

predicted = []
for n in test_set:
    p = []
    for n1 in n:
        ks1 = products.index(n1[0])
        k1 = prod_means[ks1]
        ks2 = products.index(n1[1])
        k2 = prod_means[ks2]
        if float(k1) > float(k2):
        	p.append(n1[0])
        else:
        	p.append(n1[1])
    predicted.append(p)  
    
user = []									#to calculate accuracy
count = 0
count2 = 0
for i in range(0,len(test_set)):
    for j in range(0,len(test_set[i])):
    	count2 = count2+1
        if str(predicted[i][j]) == str(test_labels[i][j]):
            count = count+1
print "the accuaracy using the average method:",(1.0 * count / count2)      

993490
3
993460
3
the accuaracy using the average method: 0.711111111111
